<a href="https://colab.research.google.com/github/naveenbreddy/Data-Science-Portfolio-Projects/blob/main/Spaceship_ML_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
! cp kaggle.json ~/.kaggle/

In [167]:
#! chmod 600 ~/.kaggle/kaggle.json

In [168]:
#!kaggle competitions download -c spaceship-titanic
import zipfile

In [169]:
#!unzip spaceship-titanic.zip -d /content/

In [170]:
#!pip install pandas-profiling

In [255]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [256]:
df = pd.read_csv("/content/train.csv")

In [257]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [337]:
df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [258]:
profile = ProfileReport(df,title = 'Spaceship Titanic')

In [259]:
#profile

In [260]:
df.info() #Looks like Transported is the prediction column - binary classification

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


#EDA

My notes:
1. Binary classification problem
2. May need to balance the data if inmbalanced
3. May need to remove columns if 

In [261]:
#nulls for homeplanet
df['HomePlanet'].isnull().sum()

201

In [262]:
df['HomePlanet'].value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [263]:
df['HomePlanet'].mode()[0]

'Earth'

In [264]:
df['Cabin'].str.split('/',expand = True)[0]

0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: 0, Length: 8693, dtype: object

In [265]:
df.head(20)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [266]:
 def df_preprocessing(df):

  #Filling Homeplanet nulls with mode
  df['HomePlanet'].fillna(df['HomePlanet'].mode()[0],inplace=True)
  
  #Cryosleep - imputing missing values with mode 
  df['CryoSleep'].fillna(df['CryoSleep'].mode()[0], inplace = True)

  #Cabin - splitting the column into three - deck, cabin_no and side
  x = df['Cabin'].str.split('/',expand = True)
      
  df.insert(3, 'deck', x[0])
  df.insert(4, 'cabin_no',x[1])
  df.insert(5, 'cabin_side',x[2])
      #imputing the missing values in the three columns with mode
  df['deck'].fillna(df['deck'].mode()[0], inplace = True)        #with mode
  df['cabin_no'].fillna(df['cabin_no'].median(), inplace = True) #with median
  df['cabin_side'].fillna(df['cabin_side'].mode()[0], inplace = True)        #with mode

  df.drop('Cabin',axis=1,inplace=True)

  #Destination - imputing with mode
  df['Destination'].fillna(df['Destination'].mode()[0], inplace = True)

  #Looking at the ProfileReport - all the numeric column - Age, RoomService, FoodCourse, ShoppingMall, Spa, VRDeck are all right skewed (except Age- left) 
  #so imputing with median
  numeric_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
  for val in numeric_col:
    df[val] = df[val].fillna(df[val].median())
  
  #VIP- imputing False
  df['VIP'] = df['VIP'].fillna('False')

  #dropping name column
  df.drop('Name', axis =1, inplace =True)

  #dropping passenger_id column
  df.drop('PassengerId', axis =1, inplace =True)  


In [267]:
copy1 = df.copy()
df_preprocessing(copy1)

In [268]:
copy1.head(10)

,HomePlanet,CryoSleep,deck,cabin_no,cabin_side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,0,P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F,0,S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A,0,S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A,0,S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F,1,S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
5,Earth,False,F,0,P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,True
6,Earth,False,F,2,S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,True
7,Earth,True,G,0,S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,0.0,True
8,Earth,False,F,3,S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,True
9,Europa,True,B,1,P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,True


In [269]:

#encoding the dataset here:
categorical_features = ['HomePlanet','Destination','cabin_side','deck']

numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

ss = StandardScaler()
copy1[numeric_features] = ss.fit_transform(copy1[numeric_features])

copy1['CryoSleep'] = copy1['CryoSleep'].astype(bool)
copy1['VIP'] = copy1['VIP'].astype(bool)
Boolean_columns = ['CryoSleep','VIP','Transported']
le = LabelEncoder()
for val in Boolean_columns:
  copy1[val] = le.fit_transform(copy1[val])


In [270]:
copy1 = pd.get_dummies(copy1, columns = categorical_features)

In [271]:
copy1.columns

Index(['CryoSleep', 'cabin_no', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'cabin_side_P',
       'cabin_side_S', 'deck_A', 'deck_B', 'deck_C', 'deck_D', 'deck_E',
       'deck_F', 'deck_G', 'deck_T'],
      dtype='object')

In [272]:
#Dividning dataset to features and predictors
X = copy1.drop('Transported', axis=1)
y = copy1['Transported']

#Setting ML pipelines

In [273]:
#creating training and validation dataset
X_train, X_test, y_val_train, y_val_test = train_test_split(X, y, test_size=0.30, random_state=50,  stratify = y)

In [274]:
pipelines = {
    'rf': make_pipeline(RandomForestClassifier()),
    'gb': make_pipeline(GradientBoostingClassifier())
}

In [275]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [276]:
grid = {
    'rf': {'randomforestclassifier__n_estimators':[100,200,300]}, 

    'gb': {'gradientboostingclassifier__n_estimators':[100,200,300]}
       }

##sample code to perform RandomizedsearchCV on the set for hyperparameter tuning

In [277]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(train_features, train_labels)

##Back to modeling

In [278]:
fit_models = {}
for algo,pipeline in pipelines.items():
  model = GridSearchCV(pipeline, grid[algo], n_jobs= -1, cv=10)
  model.fit(X_train, y_val_train)
  fit_models[algo] = model


In [279]:
print(fit_models)

{'rf': GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'randomforestclassifier__n_estimators': [100, 200,
                                                                  300]}), 'gb': GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('gradientboostingclassifier',
                                        GradientBoostingClassifier())]),
             n_jobs=-1,
             param_grid={'gradientboostingclassifier__n_estimators': [100, 200,
                                                                      300]})}


#Model evaluation

In [280]:
#evaluating models from grid
for algo,model in fit_models.items():
  y_predict = model.predict(X_test)
  print(f' acprint(f' accuracy score for {algo}: {accuracy_score(y_test,y_predict)}')
  print(f'precision score for {algo}: {precision_score(y_test,y_predict)}')
  print(f'recall score for {algo}: {recall_score(y_test,y_predict)}')curacy score for {algo}: {accuracy_score(y_test,y_predict)}')
  print(f'precision score for {algo}: {precision_score(y_test,y_predict)}')
  print(f'recall score for {algo}: {recall_score(y_test,y_predict)}')


 accuracy score for rf: 0.8044478527607362
precision score for rf: 0.833195020746888
recall score for rf: 0.7646610814927647
 accuracy score for gb: 0.8021472392638037
precision score for gb: 0.7910883856829802
recall score for gb: 0.8248286367098249


#See how model performs on test data

In [345]:
main_testdf = pd.read_csv("/content/test.csv")

In [346]:
testdf = main_testdf.copy()

In [347]:
df_preprocessing(testdf)

In [348]:
testdf.head()

,HomePlanet,CryoSleep,deck,cabin_no,cabin_side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,G,3,S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F,4,S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,Europa,True,C,0,S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0
3,Europa,False,C,1,S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0
4,Earth,False,F,5,S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0


In [349]:
#encoding the dataset here:
#encoding categorical using pd.getdummies()
categorical_features = ['HomePlanet','Destination','cabin_side','deck']
testdf = pd.get_dummies(testdf, columns = categorical_features)

#scaling numeric features
numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
ss = StandardScaler()
testdf[numeric_features] = ss.fit_transform(testdf[numeric_features])

#converting CryoSleep and VIP to bool to encode 0 and 1
testdf['CryoSleep'] = testdf['CryoSleep'].astype(bool)
testdf['VIP'] = testdf['VIP'].astype(bool)
Boolean_columns = ['CryoSleep','VIP']
le = LabelEncoder()
for val in Boolean_columns:
  testdf[val] = le.fit_transform(testdf[val])



In [350]:
testdf.head()

,CryoSleep,cabin_no,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,...,cabin_side_P,cabin_side_S,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_T
0,1,3,-0.114147,0,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1,...,0,1,0,0,0,0,0,0,1,0
1,0,4,-0.684313,0,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,1,...,0,1,0,0,0,0,0,1,0,0
2,1,0,0.170937,0,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,0,...,0,1,0,0,1,0,0,0,0,0
3,0,1,0.669832,0,-0.357339,4.121518,-0.312173,-0.104002,0.226648,0,...,0,1,0,0,1,0,0,0,0,0
4,0,5,-0.613042,0,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,1,...,0,1,0,0,0,0,0,1,0,0


In [351]:
 prediction_test = fit_models['gb'].predict(testdf)

In [352]:
submission = pd.DataFrame([main_testdf['PassengerId'],prediction_test]).T
submission.columns = ['PassengerId','Transported']

In [353]:
#converting 1,0 to True False
submission['Transported']=submission['Transported'].apply(lambda x: True if x == 1 else False)


In [356]:
submission.to_csv('Kaggle_submission.csv',header=True,index = False)

In [341]:
!kaggle competitions submit -c spaceship-titanic -m "initial_gb_model" -f "Kaggle_submission.csv"

100% 54.3k/54.3k [00:00<00:00, 84.4kB/s]
Successfully submitted to Spaceship Titanic

In [327]:
!kaggle competitions submit -c spaceship-titanic -f Kaggle_submission.csv -m "Message"

100% 76.1k/76.1k [00:00<00:00, 111kB/s]
Successfully submitted to Spaceship Titanic

In [354]:
submission.dtypes

PassengerId    object
Transported      bool
dtype: object

In [355]:
submission['Transported'].value_counts()

True     2234
False    2043
Name: Transported, dtype: int64

#See what xgboost can do on this

In [357]:
from xgboost import XGBClassifier

In [358]:
xgb_model = XGBClassifier('binary:logistic',
    colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=10, 
                             min_child_weight=1.7817, n_estimators=1500,
                             reg_alpha=4.5, reg_lambda=8.5,
                             subsample=0.5213,
                             random_state=42)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


In [363]:
xgb_pred = xgb_model.fit(X_train._get_numeric_data(), np.ravel(y_val_train, order='C')).predict(X_test._get_numeric_data())

In [365]:
print(f' accuracy score for xgb: {accuracy_score(y_test,xgb_pred)}')
print(f'precision score for  xgb: {precision_score(y_test,xgb_pred)}')
print(f'recall score for  xgb: {recall_score(y_test,xgb_pred)}')

 accuracy score for xgb: 0.799079754601227
precision score for  xgb: 0.7933085501858737
recall score for  xgb: 0.8126428027418127


In [366]:
xgb_predict_test = xgb_model.predict(testdf._get_numeric_data())

In [368]:
submission2 = pd.DataFrame([main_testdf['PassengerId'],xgb_predict_test]).T
submission2.columns = ['PassengerId','Transported']

In [370]:
submission2['Transported']=submission2['Transported'].apply(lambda x: True if x == 1 else False)

In [371]:
submission.to_csv('Kaggle_submission2.csv',index=False,header = True)